In [54]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
sns.set_theme()

In [55]:
df = pd.read_csv('../datasets/competition_dataset/dev.tsv', sep='\t')

df = df[['description', 'quality']]
df = df.drop_duplicates()
df = df.dropna().reset_index()

df

,index,description,quality
0,0,"A creamed pear wine, with an attractive tang o...",45.0
1,1,"Simple and dry, this Cabernet has modest black...",31.0
2,2,"This lovely wine captures the floral, perfumed...",35.0
3,3,"The aromas are the thing here, as so often wit...",41.0
4,4,"This is an interesting, outright strange wine ...",37.0
...,...,...,...
85010,120730,"Moët's style, with its delicious forward fruit...",59.0
85011,120732,"This is soft, young and fruity, with a dominat...",42.0
85012,120733,"Showing ripe peach, pineapple and honeysuckle ...",46.0
85013,120735,A first release from this new Walla Walla wine...,58.0


In [96]:
for i in range(100):
    if df[df.quality==i].size == 0: continue
    print(i)
    print(df[df.quality==i].iloc[0].description)
    print()

0
Clean as anyone should reasonably expect given the almost unheard-of price. It offers solid cherry and blackberry flavors, while the mouthfeel is round and totally inoffensive. Drink it now; cook with it; make a sangria with it. Just enjoy it.

7
This Châteauneuf-style wine is jammy, with black stone fruits, berries and dark chocolate, and a sweet black cherry core, but it has body and tannins to lend it weight. The winemaker suggests pairing it with cassoulet. New from Don Sebastiani & Sons.

8
Cleanly made, this plump, sweet wine offers simple pear flavors at an bargain price.

12
The grapes for this impressive Aussie-style Shiraz, from Lodi and Clarksburg, are ripe; the palate boasts has rich flavors of cherries, black raspberries, blueberries and plums. Feels succulent, with a smooth texture and a long, sweetish finish, but it's perfectly dry. At this price, you can't go wrong.

13
With its black plum, blackberry and other bold, juvenile aromas, this bargain-priced Spaniard shows

---

In [226]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVR
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords as sw
text_reg = Pipeline([
    ('count_vect', CountVectorizer(binary=True, ngram_range=(1, 1))),
    ('clf', LinearSVR(random_state=42)),
])

In [227]:
from sklearn.model_selection import cross_validate

In [228]:
%%time
scores = cross_validate(text_reg, df.description, df.quality, return_estimator=True, n_jobs=2, scoring='r2')

CPU times: user 314 ms, sys: 51.6 ms, total: 365 ms
Wall time: 15.1 s


In [229]:
scores

{'fit_time': array([4.46760082, 4.44837213, 4.37846375, 4.38792014, 4.00309467]),
 'score_time': array([0.52147436, 0.51814103, 0.52657175, 0.52258015, 0.52049804]),
 'estimator': (Pipeline(steps=[('count_vect', CountVectorizer(binary=True)),
                  ('clf', LinearSVR(random_state=42))]),
  Pipeline(steps=[('count_vect', CountVectorizer(binary=True)),
                  ('clf', LinearSVR(random_state=42))]),
  Pipeline(steps=[('count_vect', CountVectorizer(binary=True)),
                  ('clf', LinearSVR(random_state=42))]),
  Pipeline(steps=[('count_vect', CountVectorizer(binary=True)),
                  ('clf', LinearSVR(random_state=42))]),
  Pipeline(steps=[('count_vect', CountVectorizer(binary=True)),
                  ('clf', LinearSVR(random_state=42))])),
 'test_score': array([0.45438279, 0.45188445, 0.46074359, 0.44401599, 0.45474449])}

In [240]:
print(f"Mean R² score: {scores['test_score'].mean():.3f} ± {scores['test_score'].std():.3f}")

Mean R² score: 0.453 ± 0.005


---

In [231]:
len(scores['estimator'][0]['count_vect'].vocabulary_)

26802